# Before You Run

Create a shortcut in your Goole Drive to the "Attendance" folder (the folder that will hold all the files necissary for the code).




---



# Code

In [242]:
!pip install reportlab # for generating PDFs and graphics.


In [243]:
from reportlab.pdfgen.canvas import Canvas

**File check**

Before proceeding, make sure you have the following files uploaded here:
1. Font: handwritten.ttf
2. The two images answer_yes.png and answer_no.png
3. Roster from gradescope (CSE_95_Winter_2025_roster.csv)
4. Attendance Spreadsheet

Potential future work: It is not ideal to upload every time, try setting it up with drive content mount instead

In [244]:
# from google.colab import drive

# drive.mount('/content/drive', force_remount=True)

In [245]:
# cd ./drive/MyDrive/Attendance

In [246]:
import pandas as pd
import json
import numpy as np

In [247]:
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.colors import black

pdfmetrics.registerFont(TTFont('handwritten', 'handwritten.ttf'))

In [248]:
## Loads the details of students enrolled in gradescope

roster = pd.read_csv(
    "./CSE95_WI25_A00_Winter_2025_roster.csv")

roster = roster.sort_values("Email")
# roster =roster.reset_index(drop=True)
roster.head()

,First Name,Last Name,SID,Email,Role,Section
4,Anthony,Tong,A17720195,a8tong@ucsd.edu,Student,cse-95---a00-739094
6,Arnav,Dandu,A17523899,adandu@ucsd.edu,Student,cse-95---a00-739094
2,Aniruddha,Dasu,A17467014,adasu@ucsd.edu,Student,cse-95---a00-739094
1,Allison,Wang,A17845024,alw036@ucsd.edu,Student,cse-95---a00-739094
0,Alexander,Zhang,A17516108,alz004@ucsd.edu,Student,cse-95---a00-739094


In [249]:
# csv file with attendance data
attendance_file = './CSE95_WI25_Attendance.xlsx'


In [250]:
# Load attendance file into dataframe
df = pd.read_excel(attendance_file)
print(df)

               Name                Email  Present   Late
0  Alexander\tZhang      alz004@ucsd.edu     True   True
1     Allison\tWang      alw036@ucsd.edu     True   True
2   Aniruddha\tDasu       adasu@ucsd.edu     True   True
3  Anthony\tPertsel    apertsel@ucsd.edu     True   True
4     Anthony\tTong      a8tong@ucsd.edu    False   True
5      April\tHuang     aphuang@ucsd.edu     True  False
6      Arnav\tDandu      adandu@ucsd.edu    False  False
7  Andrew Smithwick  asmithwick@ucsd.edu    False   True


In [251]:
# Tanya: The change made to test if the script will work without bool values
# df["Late"][0] = "False"
# df.Late.dtypes

In [252]:
# Merge the skill demo and roster information based on email id
merged_df = pd.merge(roster, df, left_on="Email", right_on="Email", how="inner")
merged_df.head(10)

,First Name,Last Name,SID,Email,Role,Section,Name,Present,Late
0,Anthony,Tong,A17720195,a8tong@ucsd.edu,Student,cse-95---a00-739094,Anthony\tTong,False,True
1,Arnav,Dandu,A17523899,adandu@ucsd.edu,Student,cse-95---a00-739094,Arnav\tDandu,False,False
2,Aniruddha,Dasu,A17467014,adasu@ucsd.edu,Student,cse-95---a00-739094,Aniruddha\tDasu,True,True
3,Allison,Wang,A17845024,alw036@ucsd.edu,Student,cse-95---a00-739094,Allison\tWang,True,True
4,Alexander,Zhang,A17516108,alz004@ucsd.edu,Student,cse-95---a00-739094,Alexander\tZhang,True,True
5,Anthony,Pertsel,A17363270,apertsel@ucsd.edu,Student,cse-95---a00-739094,Anthony\tPertsel,True,True
6,April,Huang,A17793749,aphuang@ucsd.edu,Student,cse-95---a00-739094,April\tHuang,True,False


In [253]:
# print lengths of the 3 dataframes so far
len(merged_df), len(roster), len(df)

(7, 33, 8)

In [254]:
# check the difference between data from attendance and the merged data. Any differences signal invalid emails
# set(df["UID"]) - set(roster["UID"])

In [255]:
# Create Template
canvasTemp = Canvas("attendanceTemplate.pdf")

canvasTemp.setFont("Times-Roman", 24)
canvasTemp.setFillColorRGB(0, 0, 0)

canvasTemp.drawString(72, 800, "Name: ")

canvasTemp.setFont("Times-Roman", 24)
canvasTemp.drawString(72, 770, "PID: ")

canvasTemp.setFont("Times-Roman", 24)

canvasTemp.drawString(72, 740, "1. Active participation?")

canvasTemp.drawImage("YesNoBlank.png", 70, 660, width = 88, height = 71) # 118 x 95

canvasTemp.drawString(72, 645, "2. Was the student late to lab?")

canvasTemp.drawImage("YesNoBlank.png", 72, 565, width = 88, height = 71) # 118 x 95

canvasTemp.save()


In [256]:
# Attendance Calculation
def AttendanceCalc(fileName, studentName, PID, participated, late):
    canvas = Canvas(fileName + ".pdf")

    canvas.setFont("Times-Roman", 24)
    canvas.setFillColorRGB(0, 0, 0)

    canvas.drawString(72, 800, "Name: ")
    canvas.setFont("handwritten", 24)
    canvas.drawString(150, 800, studentName)

    canvas.setFont("Times-Roman", 24)
    canvas.drawString(72, 770, "PID: ")
    canvas.setFont("handwritten", 24)
    canvas.drawString(150, 770, PID)

    canvas.setFont("Times-Roman", 24)

    discussImage = "answer-no.png"
    lateImage = "answer-no.png"

    if(not isinstance(participated, bool)):
      raise TypeError("Values in the participated column must be booleans")

    if(not isinstance(late, bool)):
      raise TypeError("Values in the late column must be booleans")

    if participated:
      discussImage = "answer-yes.png"

    if late:
      lateImage = "answer-yes.png"

    canvas.drawString(72, 740, "1. Active participation?")

    canvas.drawImage(discussImage, 70, 660, width = 88, height = 71) # 118 x 95

    canvas.drawString(72, 645, "2. Was the student late to class?")

    canvas.drawImage(lateImage, 72, 565, width = 88, height = 71) # 118 x 95

    canvas.save()

    print(studentName)


### Generating Folder To Download On Google Drive

In [257]:
myDir = "./AttendanceWeek8"

In [258]:
!mkdir {myDir} #Outputs to google drive

for index, row in merged_df.iterrows():
  present = row["Present"]
  late = row["Late"]

  name = row["Name"] # read name data from merged_df; note that this should be in the format "<First_Name><space><Last_Name>"
  # call the function corresponding to the correct skill demo to generate student pdf files inside SDx directory.
  AttendanceCalc(myDir+"/"+row["SID"], name, row["SID"], present, late)


Anthony	Tong
Arnav	Dandu
Aniruddha	Dasu
Allison	Wang
Alexander	Zhang
Anthony	Pertsel
April	Huang


In [259]:
# zip SDx directory to enable you to download all pdfs at once from Colab
# make sure to update the command with the correct directory

!zip -r {myDir}.zip {myDir}

  adding: AttendanceWeek8/ (stored 0%)
  adding: AttendanceWeek8/A17363270.pdf (deflated 8%)
  adding: AttendanceWeek8/A17845024.pdf (deflated 8%)
  adding: AttendanceWeek8/A17516108.pdf (deflated 8%)
  adding: AttendanceWeek8/A17467014.pdf (deflated 8%)
  adding: AttendanceWeek8/A17793749.pdf (deflated 9%)
  adding: AttendanceWeek8/A17720195.pdf (deflated 9%)
  adding: AttendanceWeek8/A17523899.pdf (deflated 8%)


### Generating Folder To Download On Google Colab

In [260]:
# https://stackoverflow.com/questions/35497069/passing-ipython-variables-as-arguments-to-bash-commands
!mkdir ../../../../../content/{myDir} #Outputs to google drive

for index, row in merged_df.iterrows():
  present = row["Present"]
  late = row["Late"]

  name = row["Name"] # read name data from merged_df; note that this should be in the format "<First_Name><space><Last_Name>"
  # call the function corresponding to the correct skill demo to generate student pdf files inside SDx directory.
  AttendanceCalc("../../../../../content/"+myDir+"/"+row["SID"], name, row["SID"], present, late)


mkdir: cannot create directory ‘../../../../../content/./AttendanceWeek8’: File exists
Anthony	Tong
Arnav	Dandu
Aniruddha	Dasu
Allison	Wang
Alexander	Zhang
Anthony	Pertsel
April	Huang


In [261]:
# zip SDx directory to enable you to download all pdfs at once from Colab
# make sure to update the command with the correct directory

!zip -r ../../../../../content/AttendanceWeek8.zip ../../../../../content/AttendanceWeek8

  adding: ../../../../../content/AttendanceWeek8/ (stored 0%)
  adding: ../../../../../content/AttendanceWeek8/A17363270.pdf (deflated 8%)
  adding: ../../../../../content/AttendanceWeek8/A17845024.pdf (deflated 8%)
  adding: ../../../../../content/AttendanceWeek8/A17516108.pdf (deflated 8%)
  adding: ../../../../../content/AttendanceWeek8/A17467014.pdf (deflated 8%)
  adding: ../../../../../content/AttendanceWeek8/A17793749.pdf (deflated 9%)
  adding: ../../../../../content/AttendanceWeek8/A17720195.pdf (deflated 9%)
  adding: ../../../../../content/AttendanceWeek8/A17523899.pdf (deflated 8%)


# Grade Check

In [262]:
# # Read points from gradecope. The total score here would be on a scale of 2. 0 - try again, 1 - partial pass, 2 - full pass
# gradescope = pd.read_csv(
#     "/content/Skill_Demo_3_scores.csv")

# gradescope = gradescope[['Email', 'Total Score']]

# gradescope = gradescope.sort_values("Email")
# gradescope = gradescope.reset_index(drop=True)
# gradescope.head()

In [263]:
# def calculate_points(score):
#     if score == 1: # full pass criteria based on skill demo score
#         return 2 # means full pass
#     if score > 0.8: # partial pass criteria based on skill demo score
#         return 1 # means partial pass
#     return 0 # means no pass

# # Create a new dataframe with "EMAIL" and calculated points
# new_df = pd.DataFrame({
#     'EMAIL': merged_df['Email'],
#     'Points_Out_Of_2': merged_df['Score'].apply(calculate_points)
# })
# new_df.head()

In [264]:
# # Merge this calculated score dataframe (new_df) with gradescope score dataframe (gradescope) using email

# score_checker = pd.merge(new_df, gradescope, left_on="EMAIL", right_on="Email", how="inner")
# score_checker.head(10)

In [265]:
# # no output here => all good
# # there would be some output in case of score mismatch

# for index, row in score_checker.iterrows():
#     if row["Points_Out_Of_2"] != row["Total Score"]:
#         print("Mismatch: ", row["Email"], ", Points_out_of_2: ", row["Points_Out_Of_2"], ", Total Score: ", row["Total Score"])

In [266]:
# drive.flush_and_unmount()